## Análise da base de dados `Beta` utilizando algoritmos de ML

Neste notebook será analisado o `Beta dataset` utilizando algoritmos de ML para realizar a (1) extração de características, (2) seleção de características e (3) classificação dos dados

### Pontos importantes do dataset

- Frequências estimuladas (total de 40, com a diferença de 0.2 Hz uma da outra): 8.0, 8.2, ..., 15.6, 15.8;
- Taxa de amostragem: 250 Hz

### Analisar os "momentos" em que ocorrem evocação do sinal SSVEP

1. Criar o objeto `MNE` a partir dos dados dados do participante;
2. Aplicar no objeto `MNE` o filtro passa-faixa nos valores de 6 - 18 Hz;
3. Criar cópias do objeto `MNE` com fatias de tempo menores para analisar momentos que ocorrem estimulos ou não (verificar artigo);
    a) 0.0 - 0.5 segundos e 2.5 - 3.0 segundos ocorre apenas ruído;
    b) 0.5 - 2.5 segundos ocorre sinal SSVEP (com ruídos)
4. Com os sinais separados em objetos `MNE`, aplicar a `FFT`, para que seja possível plotar gráficos que contenham (ou não) as informações.
    - Os dados devem ser plotados no domínio da frequência (após a transformada de Fourier). O FFT pode ser realizado pela biblioteca `scipy.fft`.
    - Deve ser observado que as janelas (a) com ruído não aparecerão de fato o sinal SSVEP.

### Extração de características

Uma característica importante de acordo com o artigo base do dataset `BETA` é o *signal-to-noise ratio* (SNR).
São dois tipos de características SNR que podem ser implementadas: SNR de banda estreita (`narrow-SNR`) e SNR de banda larga (`wide-band SNR`).

Uma boa prática, é considerar o ruído das medidas de `SNR`, uma vez que os dados `SSVEP` não estão estimulados durante os períodos de 0 a 0,5 segundos e de 2,5 a 3 segundos. O ruído pode afetar a precisão das medidas de `SNR` e, portanto, é aconselhável levar isso em consideração.

Vamos realizar todos esses cálculos com dados fictícios:

+++++++++++++++++++++++++++++++++++++++++++++++++


Carregamento das bibliotecas que serão utilizadas

In [1]:
%matplotlib inline
import numpy as np
from sklearn.preprocessing import LabelEncoder
import mne
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score
from sklearn.feature_selection import RFECV

In [9]:
data = np.load("../../datasets/avi/beta/data.npy")

print(data.shape)

n_channels = 64
sfreq = 250
ch_names = list(np.load("../../datasets/avi/beta/channels.npy"))
ch_types = ['eeg'] * len(ch_names)
info = mne.create_info(ch_names, sfreq=sfreq, ch_types=ch_types)
best = ['P6', 'PO3']

labels = np.load("../../datasets/avi/beta/labels.npy")
unique_labels = sorted(set(labels))
event_dict = {str(value): index  for index, value in enumerate(unique_labels)}
labels.shape

(160, 64, 750)


(160,)

In [12]:
# método para transformar labels categóricos
le = LabelEncoder()
events = np.column_stack((
    np.array(range(len(labels))),
    np.zeros(160, dtype=int),
    le.fit_transform(labels))
)

# print(events.shape)

mne_data = mne.EpochsArray(data, info, events, event_id=event_dict)
filtered_mne_data = mne_data.filter(6, 18)

filtered_mne_data

Not setting metadata
160 matching events found
No baseline correction applied
0 projection items activated
Setting up band-pass filter from 6 - 18 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 6.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 5.00 Hz)
- Upper passband edge: 18.00 Hz
- Upper transition bandwidth: 4.50 Hz (-6 dB cutoff frequency: 20.25 Hz)
- Filter length: 413 samples (1.652 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 10240 out of 10240 | elapsed:    1.7s finished


Number of events,160
Events,10.0: 410.2: 410.4: 410.6: 410.8: 411.0: 411.2: 411.4: 411.6: 411.8: 412.0: 412.2: 412.4: 412.600000000000001: 412.8: 413.0: 413.200000000000001: 413.4: 413.600000000000001: 413.8: 414.0: 414.200000000000001: 414.4: 414.600000000000001: 414.8: 415.0: 415.200000000000001: 415.4: 415.600000000000001: 415.8: 48.0: 48.2: 48.4: 48.6: 48.799999999999999: 49.0: 49.2: 49.4: 49.6: 49.8: 4
Time range,0.000 – 2.996 s
Baseline,off


Agora iremos estimar o ruído de fundo, para calcular posteriormente o `narrow SNR` e o `wide-band SNR`. 

In [13]:
# Estimando o ruído de fundo

# intervalos de tempo sem estímulo (0 a 0,5 segundos e 2,5 a 3 segundos)
base_start = 0
base_end = 125
rest_start = 625
rest_end = 750

# armazena uma lista com as médias de potência para cada canal
noise_power = []
# consideramos a primeira amostra (1º target, 1º trial)
for channel_data in data[0, :, :]:
    fft_result = np.fft.fft(channel_data)
    # densidade espectral de potência (PSD)
    psd = np.abs(fft_result) ** 2
    # média da potência nos intervalos de tempo sem estímulo
    base_power = np.mean(psd[base_start:base_end])
    rest_power = np.mean(psd[rest_start:rest_end])
    # média das duas médias de potência obtidas anteriormente
    mean_noise_power = (base_power + rest_power) / 2
    noise_power.append(mean_noise_power)
#média das médias de potência de todos os canais para estimar o ruído de fundo
estimated_background_noise = np.mean(noise_power)
estimated_background_noise

77926.68169320165

Antes de calcular os SNRs, precisamos obter as amplitudes alvo por meio dos dados EEG:

In [14]:
from scipy.signal import find_peaks

sr = 250

# frequências alvo
target_frequencies = np.arange(8, 16, 0.2)
# lista para armazenar as amplitudes nas frequências alvo
target_amplitudes = []

for channel_data in data[0, :, :]:
    fft_result = np.fft.fft(channel_data)
    psd = np.abs(fft_result) ** 2
    frequencies = np.fft.fftfreq(len(fft_result), 1 / sr)
    target_amplitudes_trial = []
    for target_frequency in target_frequencies:
        # encontrando o índice da frequência alvo no espectro de frequência
        index = np.argmin(np.abs(frequencies - target_frequency))
        # amplitude na frequência alvo
        amplitude = np.sqrt(psd[index])
        target_amplitudes_trial.append(amplitude)
    target_amplitudes.append(target_amplitudes_trial)
target_amplitudes = np.array(target_amplitudes)
target_amplitudes.shape

(64, 40)

Vamos calcular o SNR de "banda estreita". Pode ser observado pela seguinte equação:

$SNR_{banda\ estreita} = 10 \cdot \log_{10}\left(\frac{\text{energia total do espectro}}{\text{média das amplitudes nas frequências vizinhas}}\right)$

Já o SNR de banda larga é definido da seguinte forma:

$SNR_{banda\ larga} = 10 \cdot \log_{10}\left(\frac{\text{energia total do espectro}}{\text{energia total do espectro de amplitude}}\right)$

In [5]:
# forçando (estragando) valor de "estimated_background_noise" para não sobrar valores negativos
estimated_background_noise = 1.
target_amplitudes_adjusted = target_amplitudes - estimated_background_noise

# subtraindo o ruído de fundo das amplitudes
narrow_band_SNR = 10 * np.log10(target_amplitudes_adjusted / estimated_background_noise)
print(narrow_band_SNR)
print(narrow_band_SNR.shape)

total_power = np.sum(target_amplitudes_adjusted)
wide_band_SNR = 10 * np.log10(target_amplitudes_adjusted / total_power)
print(wide_band_SNR)
print(wide_band_SNR.shape)


[[22.51648602 22.14875618 22.14875618 ... 24.29600335 23.6699112
  23.6699112 ]
 [23.97809677 24.54432243 24.54432243 ... 25.68077667 22.94453944
  22.94453944]
 [23.05740352 21.05731764 21.05731764 ... 24.53244202 23.52492048
  23.52492048]
 ...
 [19.26282037 25.91372478 25.91372478 ... 22.31791424 15.64358515
  15.64358515]
 [26.31550064 21.25065785 21.25065785 ... 21.61253012 25.76433945
  25.76433945]
 [25.95941299 15.02203279 15.02203279 ... 25.23254276 19.43718037
  19.43718037]]
(64, 40)
[[-35.43340772 -35.80113755 -35.80113755 ... -33.65389038 -34.27998253
  -34.27998253]
 [-33.97179696 -33.4055713  -33.4055713  ... -32.26911706 -35.00535429
  -35.00535429]
 [-34.89249022 -36.89257609 -36.89257609 ... -33.41745171 -34.42497325
  -34.42497325]
 ...
 [-38.68707336 -32.03616896 -32.03616896 ... -35.63197949 -42.30630859
  -42.30630859]
 [-31.6343931  -36.69923589 -36.69923589 ... -36.33736361 -32.18555428
  -32.18555428]
 [-31.99048074 -42.92786095 -42.92786095 ... -32.71735098 -3

Remoção manual de caracteristicas

In [15]:
ch_names = np.load("../../datasets//avi/beta/channels.npy")

channels_to_keep = ['PZ', 'PO3', 'PO5', 'PO4', 'PO6', 'POZ', 'O1', 'OZ', 'O2']
indexes_to_remove = np.where(np.isin(ch_names, channels_to_keep, invert=True))

print("Before", data.shape)
data_filtered = np.delete(data, indexes_to_remove, axis=1)
print("After", data_filtered.shape)
print("Labels", labels.shape)

Before (160, 64, 750)
After (160, 9, 750)
Labels (160,)


In [16]:
data_filtered_reshape = data_filtered.reshape(data_filtered.shape[0], data_filtered.shape[1] * data_filtered.shape[2])
print(data_filtered_reshape.shape)

X = StandardScaler().fit_transform(data_filtered_reshape)
y = LabelEncoder().fit_transform(labels)

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

clf = SVC(kernel='linear', C=1, random_state=42, probability=True)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)

print("accuracy", accuracy_score(y_test, y_pred))
print("f1_score", f1_score(y_test, y_pred, average="weighted"))

(160, 6750)
accuracy 0.2708333333333333
f1_score 0.2490079365079365


Remoção automática RFE

In [17]:
X_reshape_auto = data.reshape(data.shape[0], data.shape[1] * data.shape[2])
print(X_reshape_auto.shape)

(160, 48000)


In [20]:
X_reshape_auto = StandardScaler().fit_transform(X_reshape_auto)

rfe = RFECV(SVC(kernel="linear"), step=0.0001, min_features_to_select=1, cv=3)
X_final = rfe.fit_transform(X_reshape_auto, y)
print(X_final.shape)

In [19]:
x_train, x_test, y_train, y_test = train_test_split(X_final, y, test_size=0.3, random_state=42)

clf = SVC(kernel='linear', C=1, random_state=42, probability=True)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)

print("accuracy", accuracy_score(y_test, y_pred))
print("f1_score", f1_score(y_test, y_pred, average="weighted"))


NameError: name 'X_final' is not defined

Ao final desta etapa, será obtido um vetor de características. Estas podem ser:
- `narrow SNR` (brigatória);
- `wide-band SNR` (brigatória);
- Maior valor espectral (FFT);
- Média dos valores espectrais (FFT).

Dimensionalidade dos dados será explicada da seguinte forma:

`40, 4, 64, 750` -> 40 targets, 4 trials, 64 canais e 750 valores
`160, 64 (SNR) + 64 (média) + 64 (maior) ...`
Resultando em `160, 192`.

### Seleção de características e classificação

Como existem diversos eletrodos (canais) que não obtém sinal SSVEP, podemos extrair as caracteríscas que não contribuem para a classificação dos dados.

Podemos utilizar o método `RFE` (*Recursive Feature Elimination*) aplicado por meio de `sklearn.feature_selection.RFE`, aprimorando o parâmetro `n_features_to_select` até obter o melhor resultado de classificação.

Para a classificação propriamente dita, é considerado o uso do método `SVM`.